# Prepare the data.

## Wikipedia

Link: 

## Oscar
Link: 

## Books

Polish wolne lektruy

Usefull shell commands


Cat all text files and instert new line between each text

```sh
find *.txt | xargs -I{} sh -c "cat {}; echo ''" > all_contents.txt

```

Take 11768022 first lines form splited wikipedia file
```
head -11768022 corpus_wiki_books_non_word_tokenized_2020-02-13.txt > corpus_wiki_lines_non_tok_2020-02-13.txt
```




In [ ]:
#
import csv

wikipedia_path='./data/wikipedia_2020-02-01.csv'

with open(wikipedia_path) as csv_file:
    csv_reader = csv.DictReader(csv_file)

    for article in csv_reader:
        text = article['CleanText']

    wiki_text_gen = get_tokenized_text(data=csv_reader, row_label='CleanText')
    for i, article in enumerate(wiki_text_gen):
        output_file.write(article)
        output_file.write('\n')



for row in data:
        text = row[row_label]
        sentences = sent_tokenize(text, 'polish')
        text = ''
        for sentence in sentences:
            tokenized_sentence = ' '.join(word_tokenize(sentence))
            text = '\n'.join([text, tokenized_sentence])
        yield text        
        

## Train the BPE sentencepiece model

We used the [sentencepiece](https://github.com/google/sentencepiece)


In [1]:
import sentencepiece as spm
import datetime